In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from bayes_opt_adaptive  import BayesianOptimization
from bayes_opt_adaptive import UtilityFunction
from scipy.optimize import minimize, NonlinearConstraint, Bounds, SR1
from bayes_opt_adaptive.logger import JSONLogger
from bayes_opt_adaptive.event import Events
from bayes_opt_adaptive.util import load_logs
from cases import Case1,Case2,Case3
import numpy as np
import math
import xlwt
from pathlib import Path
import subprocess
import os
import win32com.client as win32
import time
import random
import subprocess

In [4]:

BASE_DIR = Path().absolute()
DIR = BASE_DIR/'Results'
file_location = BASE_DIR/'extractive_distillation'

In [5]:
*

SyntaxError: invalid syntax (796671574.py, line 1)

In [ ]:
sol = 'ASBO'
case_name = 'Case4'
runs_number = 10

flowsheet_pass = 0  
list_realvalue = []
def Case4(x):
    global Application, flowsheet_pass,list_realvalue  
    x = [round(num, 3) for num in x]
    for i in range(len(x)):
        globals()[f'x{i}'] = x[i]
    max_attempts = 1
    attempts = 0
    result = None
    try:
        while attempts < max_attempts:
            try:
                Application.Tree.FindNode("\Data\Streams\SOLVENT\Input\TOTFLOW\MIXED").Value = x0
                Application.Tree.FindNode("\Data\Blocks\COLUMN\Input\BASIS_D").Value = x1
                Application.Tree.FindNode("\Data\Blocks\COLUMN\Input\BASIS_RR").Value = x2 
                Application.Tree.FindNode("\Data\Blocks\COL-REC\Input\BASIS_D").Value = x3
                Application.Tree.FindNode("\Data\Blocks\COL-REC\Input\BASIS_RR").Value = x4
                Application.Reinit()
                Application.Engine.Run2(1)
                while Application.Engine.IsRunning == 1:
                    time.sleep(1)
                status_code = Application.Tree.FindNode("\Data\Results Summary\Run-Status").AttributeValue(12)
                if (status_code & 1) == 1:     
                    status =  'Available'
                elif (status_code & 4) == 4:   
                    status =  'Warning'
                elif (status_code & 32) == 32:  
                    status =  'Error'
                else:
                    status =  'Error'
                if status == 'Available' or status == 'Warning':
                    column_con_duty =Application.Tree.FindNode("\Data\Blocks\COLUMN\Output\COND_COST").Value
                    column_reb_duty = Application.Tree.FindNode("\Data\Blocks\COLUMN\Output\REB_COST").Value
                    rec_con_duty = Application.Tree.FindNode("\Data\Blocks\COL-REC\Output\COND_COST").Value
                    rec_reb_duty = Application.Tree.FindNode("\Data\Blocks\COL-REC\Output\REB_COST").Value
                    obj = column_con_duty+rec_con_duty+column_reb_duty+rec_reb_duty
                    purity_heptan = Application.Tree.FindNode("\Data\Streams\C7\Output\MOLEFRAC\MIXED\\N-HEPTAN").Value
                    purity_tolue = Application.Tree.FindNode("\Data\Streams\TOLUENE\Output\MOLEFRAC\MIXED\TOLUE-01").Value
                    result = -obj,[purity_heptan,purity_tolue]
                    # print('constraint',purity_heptan,purity_tolue,obj)
                    break
                else:
                    raise Exception("Convergence failed")
            except Exception as e:
                # print(f"Run failed: {e}")
                attempts += 1
                for i in range(len(x)):
                    epsilon = 0.01*int("%d" % attempts)*x[i]
                    x[i] = x[i] + random.choice((-1,1))* epsilon
                # print(f"Number of restarts: {attempts}/{max_attempts}")

    except Exception as e: 
        print("Exception: ", e.args) 
        print("Aspen error, try to close Aspen!")
        Application.Close()

    if result is not None:
        result = result
    else:
        result = -10000,[0,0]
    flowsheet_pass += 1
    return result

def obj_fun(x0,x1,x2,x3,x4):
    x = [x0,x1,x2,x3,x4,]
    res = Case4(x)
    return res

nc_lb = [0.98,0.98]
nc_ub = [1,   1 ]
cons = NonlinearConstraint(obj_fun, nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.0001)

pbounds = {'x0':(40,100),'x1': (30, 80),'x2': (3,10), 'x3': (20,80),'x4': (1,5)}
for i in range(runs_number):
    
    version = "37.0"
    Application = win32.Dispatch('Apwn.Document.' + version)

    # 打开Aspen文件
    # 将python文件所在文件夹和aspen文件所在文件夹放在同一个文件夹下
    path = os.path.join(file_location/f'extractive_distillation.bkp')
    Application.InitFromArchive2(path)  # 打开文件

    # 设置AP用户界面的可见性，1为可见，0为不可见
    Application.Visible = 1

    # 压制对话框的弹出，1为压制；0为不压制
    Application.SuppressDialogs = 1
    optimizer = BayesianOptimization(f=obj_fun, pbounds=pbounds,constraint=cons,verbose=0,random_state=i,relax_margin=1e-3,)
    acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)
    optimizer.maximize(init_points=100, n_iter=60,p_hyper=[5, 2, 0.78, 0.95,1e-5],acquisition_function=acquisition_function,allow_duplicate_points=True)
    print(optimizer.max)
    subprocess.run('taskkill /F /IM AspenPlus.exe')
    subprocess.run('taskkill /F /IM AspenPlus.exe')

{'target': -261.0456178, 'params': {'x0': 100.0, 'x1': 50.376764638972894, 'x2': 4.699694647385, 'x3': 27.84727012748006, 'x4': 1.0000000000000029}, 'constraint': array([0.98279178, 0.98713968])}
{'target': -262.3285101, 'params': {'x0': 58.46551247331177, 'x1': 50.68922811261341, 'x2': 4.8016933542143505, 'x3': 24.05820749234631, 'x4': 1.0000000000000002}, 'constraint': array([0.98134793, 0.99137652])}
{'target': -261.08086979999996, 'params': {'x0': 70.57545878163675, 'x1': 50.03761988612085, 'x2': 4.6825467677192245, 'x3': 29.64578524367778, 'x4': 1.0}, 'constraint': array([0.98680172, 0.98185141])}
{'target': -256.4105508, 'params': {'x0': 60.028940755916125, 'x1': 50.465672744538786, 'x2': 4.559908683112119, 'x3': 28.859025488898276, 'x4': 1.0000000000000002}, 'constraint': array([0.98063884, 0.98426979])}
{'target': -259.6177765, 'params': {'x0': 76.8985176302122, 'x1': 50.133458352681615, 'x2': 4.674076404936232, 'x3': 28.534956822236843, 'x4': 1.0}, 'constraint': array([0.98627

In [ ]:
sol = 'ASBO'
case_name = 'Case4'
runs_number = 10

flowsheet_pass = 0  
list_realvalue = []
def Case4(x):
    global Application, flowsheet_pass,list_realvalue  
    x = [round(num, 3) for num in x]
    for i in range(len(x)):
        globals()[f'x{i}'] = x[i]
    max_attempts = 1
    attempts = 0
    result = None
    try:
        while attempts < max_attempts:
            try:
                Application.Tree.FindNode("\Data\Streams\SOLVENT\Input\TOTFLOW\MIXED").Value = x0
                Application.Tree.FindNode("\Data\Blocks\COLUMN\Input\BASIS_D").Value = x1
                Application.Tree.FindNode("\Data\Blocks\COLUMN\Input\BASIS_RR").Value = x2 
                Application.Tree.FindNode("\Data\Blocks\COL-REC\Input\BASIS_D").Value = x3
                Application.Tree.FindNode("\Data\Blocks\COL-REC\Input\BASIS_RR").Value = x4
                Application.Reinit()
                Application.Engine.Run2(1)
                while Application.Engine.IsRunning == 1:
                    time.sleep(1)
                status_code = Application.Tree.FindNode("\Data\Results Summary\Run-Status").AttributeValue(12)
                if (status_code & 1) == 1:     
                    status =  'Available'
                elif (status_code & 4) == 4:   
                    status =  'Warning'
                elif (status_code & 32) == 32:  
                    status =  'Error'
                else:
                    status =  'Error'
                if status == 'Available' or status == 'Warning':
                    column_con_duty =Application.Tree.FindNode("\Data\Blocks\COLUMN\Output\COND_COST").Value
                    column_reb_duty = Application.Tree.FindNode("\Data\Blocks\COLUMN\Output\REB_COST").Value
                    rec_con_duty = Application.Tree.FindNode("\Data\Blocks\COL-REC\Output\COND_COST").Value
                    rec_reb_duty = Application.Tree.FindNode("\Data\Blocks\COL-REC\Output\REB_COST").Value
                    obj = column_con_duty+rec_con_duty+column_reb_duty+rec_reb_duty
                    purity_heptan = Application.Tree.FindNode("\Data\Streams\C7\Output\MOLEFRAC\MIXED\\N-HEPTAN").Value
                    purity_tolue = Application.Tree.FindNode("\Data\Streams\TOLUENE\Output\MOLEFRAC\MIXED\TOLUE-01").Value
                    result = -obj,[purity_heptan,purity_tolue]
                    # print('constraint',purity_heptan,purity_tolue,obj)
                    break
                else:
                    raise Exception("Convergence failed")
            except Exception as e:
                # print(f"Run failed: {e}")
                attempts += 1
                for i in range(len(x)):
                    epsilon = 0.01*int("%d" % attempts)*x[i]
                    x[i] = x[i] + random.choice((-1,1))* epsilon
                # print(f"Number of restarts: {attempts}/{max_attempts}")

    except Exception as e: 
        print("Exception: ", e.args) 
        print("Aspen error, try to close Aspen!")
        Application.Close()

    if result is not None:
        result = result
    else:
        result = -10000,[0,0]
    flowsheet_pass += 1
    return result

def obj_fun(x0,x1,x2,x3,x4):
    x = [x0,x1,x2,x3,x4,]
    res = Case4(x)
    return res

nc_lb = [0.98,0.98]
nc_ub = [1,   1 ]
cons = NonlinearConstraint(obj_fun, nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.0001)

pbounds = {'x0':(40,100),'x1': (30, 80),'x2': (3,10), 'x3': (20,80),'x4': (1,5)}
for i in range(runs_number):
    
    version = "37.0"
    Application = win32.Dispatch('Apwn.Document.' + version)

    # 打开Aspen文件
    # 将python文件所在文件夹和aspen文件所在文件夹放在同一个文件夹下
    path = os.path.join(file_location/f'extractive_distillation.bkp')
    Application.InitFromArchive2(path)  # 打开文件

    # 设置AP用户界面的可见性，1为可见，0为不可见
    Application.Visible = 1

    # 压制对话框的弹出，1为压制；0为不压制
    Application.SuppressDialogs = 1
    optimizer = BayesianOptimization(f=obj_fun, pbounds=pbounds,constraint=cons,verbose=0,random_state=i,relax_margin=1e-3,)
    acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)
    optimizer.maximize(init_points=100, n_iter=60,p_hyper=[5, 2, 0.78, 0.95,1e-5],acquisition_function=acquisition_function,allow_duplicate_points=True)
    print(optimizer.max)
    subprocess.run('taskkill /F /IM AspenPlus.exe')
    subprocess.run('taskkill /F /IM AspenPlus.exe')

In [ ]:
*

SyntaxError: invalid syntax (796671574.py, line 1)

In [ ]:
sol = 'slsqp'
case_name = 'Case4'
runs_number = 10
initial_x = []
fun_r = []
nfev_r = []
success_r = []
message_r = []

flowsheet_pass = 0  
list_realvalue = []

def Case4(x):
    global Application, flowsheet_pass, func_count,list_realvalue  
    #去归一化
    def unnormalization(data,max,min): 
        res = (max - min) * data + min 
        return round(res, 5) 
    lb = [40,  30,   3,   20,  1] 
    ub = [100, 80,  10,   80,  5]
    x0 = unnormalization(x[0], ub[0], lb[0])
    x1 = unnormalization(x[1], ub[1], lb[1])
    x2 = unnormalization(x[2], ub[2], lb[2])
    x3 = unnormalization(x[3], ub[3], lb[3])
    x4 = unnormalization(x[4], ub[4], lb[4])
    max_attempts = 1
    attempts = 0
    result = None
    try:
        while attempts < max_attempts:
            try:
                Application.Tree.FindNode("\Data\Streams\SOLVENT\Input\TOTFLOW\MIXED").Value = x0
                Application.Tree.FindNode("\Data\Blocks\COLUMN\Input\BASIS_D").Value = x1
                Application.Tree.FindNode("\Data\Blocks\COLUMN\Input\BASIS_RR").Value = x2 
                Application.Tree.FindNode("\Data\Blocks\COL-REC\Input\BASIS_D").Value = x3
                Application.Tree.FindNode("\Data\Blocks\COL-REC\Input\BASIS_RR").Value = x4
                Application.Reinit()
                Application.Engine.Run2(1)
                while Application.Engine.IsRunning == 1:
                    time.sleep(1)
                status_code = Application.Tree.FindNode("\Data\Results Summary\Run-Status").AttributeValue(12)
                if (status_code & 1) == 1:     
                    status =  'Available'
                elif (status_code & 4) == 4:   
                    status =  'Warning'
                elif (status_code & 32) == 32:  
                    status =  'Error'
                else:
                    status =  'Error'
                if status == 'Available' or status == 'Warning':
                    column_con_duty =Application.Tree.FindNode("\Data\Blocks\COLUMN\Output\COND_COST").Value
                    column_reb_duty = Application.Tree.FindNode("\Data\Blocks\COLUMN\Output\REB_COST").Value
                    rec_con_duty = Application.Tree.FindNode("\Data\Blocks\COL-REC\Output\COND_COST").Value
                    rec_reb_duty = Application.Tree.FindNode("\Data\Blocks\COL-REC\Output\REB_COST").Value
                    obj = column_con_duty+rec_con_duty+column_reb_duty+rec_reb_duty
                    purity_heptan = Application.Tree.FindNode("\Data\Streams\C7\Output\MOLEFRAC\MIXED\\N-HEPTAN").Value
                    purity_tolue = Application.Tree.FindNode("\Data\Streams\TOLUENE\Output\MOLEFRAC\MIXED\TOLUE-01").Value
                    result = -obj,purity_heptan,purity_tolue
                    print('constraint',purity_heptan,purity_tolue,obj)
                    break
                else:
                    raise Exception("Convergence failed")
            except Exception as e:
                print(f"Run failed: {e}")
                attempts += 1
                for i in range(len(x)):
                    epsilon = 0.01*int("%d" % attempts)*x[i]
                    x[i] = x[i] + random.choice((-1,1))* epsilon
                # print(f"Number of restarts: {attempts}/{max_attempts}")
    except Exception as e: 
        print("Exception: ", e.args) 
        print("Aspen error, try to close Aspen!")
        Application.Close()

    if result is not None:
        result = result
    else:
        result = -10000,0,0
    flowsheet_pass += 1
    return result


def recyc(x):
    obj_fun = lambda x: Case4(x)[0]
    nc_lb = [0.98,0.98]
    nc_ub = [1,   1]
    cons = NonlinearConstraint(lambda x: Case4(x)[1:3], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    x0 =x # [0.436 , 0.026 , 0.55 , 0.435 , 0.42]

    lb=[0,0,0,0,0]
    ub=[1,1,1,1,1]
    bd = Bounds(lb, ub, True)  

    result = minimize(obj_fun, x0, method='slsqp', bounds=bd, constraints=cons)
    print('完成一次计算',result.fun,result.x)
    initial_x.append(result.x)
    fun_r.append(result.fun)
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)
    f = xlwt.Workbook('encoding = utf-8') 
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True) 
    for i in range(len(initial_x)):
        x_values = initial_x[i].tolist()
        sheet1.write(i,0,x_values[0]) 
        sheet1.write(i,1,x_values[1])
        sheet1.write(i,2,fun_r[i])
        sheet1.write(i,3,nfev_r[i])
        sheet1.write(i,4,success_r[i])
        sheet1.write(i,5,message_r[i])
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')
    return result.x

import numpy as np
import random
random_state = np.random.RandomState(0)
def lhs_sample(n_samples=10):
    bounds_values = bounds=[[0,1],[0,1],[0,1],[0,1],[0,1]]
    n_dimensions = len(bounds_values)
    # Initialize an empty array for LHS
    result = np.empty((n_samples, n_dimensions))
    for i in range(n_dimensions):
        lower, upper = bounds_values[i]
        # Generate LHS intervals
        interval = np.linspace(lower, upper, n_samples + 1)
        points = (interval[:-1] + interval[1:]) / 2
        random_state.shuffle(points)
        result[:, i] = points
    return result
def sample(n_samples):
    # Assuming n_samples=1 for single sample generation
    lhs_data = lhs_sample(n_samples=n_samples)
    return lhs_data
from itertools import product
for combination in product(sample(10)):
    subprocess.run('taskkill /F /IM AspenPlus.exe')
    version = "37.0"
    Application = win32.Dispatch('Apwn.Document.' + version)
    path = os.path.join(file_location/f'extractive_distillation.bkp')
    print(path)
    Application.InitFromArchive2(path)  
    Application.Visible = 1
    Application.SuppressDialogs = 1
    recyc(combination)

c:\Users\fan_shuyuan\Desktop\科研ing\工艺流程优化\HPBO-V5\extractive_distillation\extractive_distillation.bkp


C:\Users\fan_shuyuan\AppData\Local\Temp\ipykernel_57340\2888243411.py:94: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(obj_fun, x0, method='slsqp', bounds=bd, constraints=cons)
D:\something\anaconda\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "


Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
Run failed: Convergence failed
完成一次计算 -10000 [0.24747525 0.35346465 0.24747525 0.66969565 0.55544445]
c:\Users\fan_shuyuan\Desktop\科研ing\工艺流程优化\HPBO-V5\extractive_distillation\extractive_distillation.bkp
constraint 0.869070518 0.99908237 363.95594760000006
constraint 0.869070017 0.999081519 363.9569108
constraint 0.868657222 0.999086284 364.1091656
constraint 0.869071581 0.999084341 364.0851083
constraint 0.

In [ ]:
sol = 'SQP'
case_name = 'Case4'
runs_number = 10

flowsheet_pass = 0  
list_realvalue = []

def Case4(x):
    global Application, flowsheet_pass,list_realvalue  
    x = [round(num, 3) for num in x]
    for i in range(len(x)):
        globals()[f'x{i}'] = x[i]
    max_attempts = 1
    attempts = 0
    result = None
    try:
        while attempts < max_attempts:
            try:
                Application.Tree.FindNode("\Data\Streams\SOLVENT\Input\TOTFLOW\MIXED").Value = x0
                Application.Tree.FindNode("\Data\Blocks\COLUMN\Input\BASIS_D").Value = x1
                Application.Tree.FindNode("\Data\Blocks\COLUMN\Input\BASIS_RR").Value = x2 
                Application.Tree.FindNode("\Data\Blocks\COL-REC\Input\BASIS_D").Value = x3
                Application.Tree.FindNode("\Data\Blocks\COL-REC\Input\BASIS_RR").Value = x4
                Application.Reinit()
                Application.Engine.Run2(1)
                while Application.Engine.IsRunning == 1:
                    time.sleep(1)
                status_code = Application.Tree.FindNode("\Data\Results Summary\Run-Status").AttributeValue(12)
                if (status_code & 1) == 1:     
                    status =  'Available'
                elif (status_code & 4) == 4:   
                    status =  'Warning'
                elif (status_code & 32) == 32:  
                    status =  'Error'
                else:
                    status =  'Error'
                if status == 'Available' or status == 'Warning':
                    column_con_duty =Application.Tree.FindNode("\Data\Blocks\COLUMN\Output\COND_COST").Value
                    column_reb_duty = Application.Tree.FindNode("\Data\Blocks\COLUMN\Output\REB_COST").Value
                    rec_con_duty = Application.Tree.FindNode("\Data\Blocks\COL-REC\Output\COND_COST").Value
                    rec_reb_duty = Application.Tree.FindNode("\Data\Blocks\COL-REC\Output\REB_COST").Value
                    obj = column_con_duty+rec_con_duty+column_reb_duty+rec_reb_duty
                    purity_heptan = Application.Tree.FindNode("\Data\Streams\C7\Output\MOLEFRAC\MIXED\\N-HEPTAN").Value
                    purity_tolue = Application.Tree.FindNode("\Data\Streams\TOLUENE\Output\MOLEFRAC\MIXED\TOLUE-01").Value
                    result = -obj,[purity_heptan,purity_tolue]
                    # print('constraint',purity_heptan,purity_tolue,obj)
                    break
                else:
                    raise Exception("Convergence failed")
            except Exception as e:
                # print(f"Run failed: {e}")
                attempts += 1
                for i in range(len(x)):
                    epsilon = 0.01*int("%d" % attempts)*x[i]
                    x[i] = x[i] + random.choice((-1,1))* epsilon
                # print(f"Number of restarts: {attempts}/{max_attempts}")

    except Exception as e: 
        # print("Exception: ", e.args) 
        # print("Aspen error, try to close Aspen!")
        Application.Close()

    if result is not None:
        result = result
    else:
        result = -10000,[0,0]
    flowsheet_pass += 1
    return result

def obj_fun(x):
    x = [x0,x1,x2,x3,x4,]
    res = Case4(x)
    return res

random_state = np.random.RandomState(2)
def lhs_sample(n_samples=1):
    bounds_values = bounds=[[40,100],[30,80],[3,10],[20,80],[1,5]]
    n_dimensions = len(bounds_values)
    # Initialize an empty array for LHS
    result = np.empty((n_samples, n_dimensions))
    
    for i in range(n_dimensions):
        lower, upper = bounds_values[i]
        # Generate LHS intervals
        interval = np.linspace(lower, upper, n_samples + 1)
        points = (interval[:-1] + interval[1:]) / 2
        random_state.shuffle(points)
        result[:, i] = points
    return result
def sample(n_samples):
    # Assuming n_samples=1 for single sample generation
    lhs_data = lhs_sample(n_samples=n_samples)
    return lhs_data

from itertools import product
for combination in product(sample(10)):
    subprocess.run('taskkill /F /IM AspenPlus.exe')
    subprocess.run('taskkill /F /IM AspenPlus.exe')
    version = "37.0"
    Application = win32.Dispatch('Apwn.Document.' + version)
    path = os.path.join(file_location/f'extractive_distillation_SQP.bkp')
    print(combination,path)
    Application.InitFromArchive2(path)  
    Application.Visible = 1
    Application.SuppressDialogs = 1
    res = obj_fun(combination)
    print(res)


(array([67.  , 37.5 ,  6.15, 53.  ,  3.2 ]),)
c:\Users\fan_shuyuan\Desktop\科研ing\工艺流程优化\HPBO-V5\extractive_distillation\extractive_distillation_SQP.bkp
(-10000, [0, 0])
(array([49.  , 62.5 ,  9.65, 77.  ,  2.4 ]),)
c:\Users\fan_shuyuan\Desktop\科研ing\工艺流程优化\HPBO-V5\extractive_distillation\extractive_distillation_SQP.bkp
(-10000, [0, 0])
(array([73.  , 32.5 ,  6.85, 71.  ,  4.4 ]),)
c:\Users\fan_shuyuan\Desktop\科研ing\工艺流程优化\HPBO-V5\extractive_distillation\extractive_distillation_SQP.bkp
(-10000, [0, 0])
(array([43.  , 42.5 ,  3.35, 23.  ,  3.6 ]),)
c:\Users\fan_shuyuan\Desktop\科研ing\工艺流程优化\HPBO-V5\extractive_distillation\extractive_distillation_SQP.bkp
(-10000, [0, 0])
(array([85.  , 47.5 ,  8.95, 35.  ,  4.  ]),)
c:\Users\fan_shuyuan\Desktop\科研ing\工艺流程优化\HPBO-V5\extractive_distillation\extractive_distillation_SQP.bkp
(-10000, [0, 0])
(array([55.  , 77.5 ,  4.05, 29.  ,  1.2 ]),)
c:\Users\fan_shuyuan\Desktop\科研ing\工艺流程优化\HPBO-V5\extractive_distillation\extractive_distillation_SQP.bkp
(-1

KeyboardInterrupt: 

In [ ]:
sol = 'BOBYQA'
case_name = 'Case4'
runs_number = 10

flowsheet_pass = 0  
list_realvalue = []

def Case4(x):
    global Application, flowsheet_pass,list_realvalue  
    x = [round(num, 3) for num in x]
    for i in range(len(x)):
        globals()[f'x{i}'] = x[i]
    max_attempts = 1
    attempts = 0
    result = None
    try:
        while attempts < max_attempts:
            try:
                Application.Tree.FindNode("\Data\Streams\SOLVENT\Input\TOTFLOW\MIXED").Value = x0
                Application.Tree.FindNode("\Data\Blocks\COLUMN\Input\BASIS_D").Value = x1
                Application.Tree.FindNode("\Data\Blocks\COLUMN\Input\BASIS_RR").Value = x2 
                Application.Tree.FindNode("\Data\Blocks\COL-REC\Input\BASIS_D").Value = x3
                Application.Tree.FindNode("\Data\Blocks\COL-REC\Input\BASIS_RR").Value = x4
                Application.Reinit()
                Application.Engine.Run2(1)
                while Application.Engine.IsRunning == 1:
                    time.sleep(1)
                status_code = Application.Tree.FindNode("\Data\Results Summary\Run-Status").AttributeValue(12)
                if (status_code & 1) == 1:     
                    status =  'Available'
                elif (status_code & 4) == 4:   
                    status =  'Warning'
                elif (status_code & 32) == 32:  
                    status =  'Error'
                else:
                    status =  'Error'
                if status == 'Available' or status == 'Warning':
                    column_con_duty =Application.Tree.FindNode("\Data\Blocks\COLUMN\Output\COND_COST").Value
                    column_reb_duty = Application.Tree.FindNode("\Data\Blocks\COLUMN\Output\REB_COST").Value
                    rec_con_duty = Application.Tree.FindNode("\Data\Blocks\COL-REC\Output\COND_COST").Value
                    rec_reb_duty = Application.Tree.FindNode("\Data\Blocks\COL-REC\Output\REB_COST").Value
                    obj = column_con_duty+rec_con_duty+column_reb_duty+rec_reb_duty
                    purity_heptan = Application.Tree.FindNode("\Data\Streams\C7\Output\MOLEFRAC\MIXED\\N-HEPTAN").Value
                    purity_tolue = Application.Tree.FindNode("\Data\Streams\TOLUENE\Output\MOLEFRAC\MIXED\TOLUE-01").Value
                    result = -obj,[purity_heptan,purity_tolue]
                    # print('constraint',purity_heptan,purity_tolue,obj)
                    break
                else:
                    raise Exception("Convergence failed")
            except Exception as e:
                print(f"Run failed: {e}")
                attempts += 1
                for i in range(len(x)):
                    epsilon = 0.01*int("%d" % attempts)*x[i]
                    x[i] = x[i] + random.choice((-1,1))* epsilon
                print(f"Number of restarts: {attempts}/{max_attempts}")

    except Exception as e: 
        print("Exception: ", e.args) 
        print("Aspen error, try to close Aspen!")
        Application.Close()

    if result is not None:
        result = result
    else:
        result = -10000,[0,0]
    flowsheet_pass += 1
    return result

def obj_fun(x0,x1,x2,x3,x4):
    x = [x0,x1,x2,x3,x4,]
    res = Case4(x)
    return res

random_state = np.random.RandomState(2)
def lhs_sample(n_samples=1):
    bounds_values = bounds=[[40,100],[30,80],[3,10],[20,80],[1,5]]
    n_dimensions = len(bounds_values)
    # Initialize an empty array for LHS
    result = np.empty((n_samples, n_dimensions))
    
    for i in range(n_dimensions):
        lower, upper = bounds_values[i]
        # Generate LHS intervals
        interval = np.linspace(lower, upper, n_samples + 1)
        points = (interval[:-1] + interval[1:]) / 2
        random_state.shuffle(points)
        result[:, i] = points
    return result
def sample(n_samples):
    # Assuming n_samples=1 for single sample generation
    lhs_data = lhs_sample(n_samples=n_samples)
    return lhs_data

from itertools import product
for combination in product(sample(10)):
    subprocess.run('taskkill /F /IM AspenPlus.exe')
    subprocess.run('taskkill /F /IM AspenPlus.exe')
    version = "37.0"
    Application = win32.Dispatch('Apwn.Document.' + version)
    path = os.path.join(file_location/f'extractive_distillation_SQP.bkp')
    print(combination,path)
    Application.InitFromArchive2(path)  
    Application.Visible = 1
    Application.SuppressDialogs = 1
    res = obj_fun(combination)
    print(res)

c:\Users\fan_shuyuan\Desktop\科研ing\工艺流程优化\HPBO-V5\extractive_distillation\extractive_distillation_BOBYQA.bkp
{'target': None, 'params': None, 'constraint': None}
c:\Users\fan_shuyuan\Desktop\科研ing\工艺流程优化\HPBO-V5\extractive_distillation\extractive_distillation_BOBYQA.bkp
{'target': None, 'params': None, 'constraint': None}
c:\Users\fan_shuyuan\Desktop\科研ing\工艺流程优化\HPBO-V5\extractive_distillation\extractive_distillation_BOBYQA.bkp
{'target': None, 'params': None, 'constraint': None}
c:\Users\fan_shuyuan\Desktop\科研ing\工艺流程优化\HPBO-V5\extractive_distillation\extractive_distillation_BOBYQA.bkp
{'target': None, 'params': None, 'constraint': None}
c:\Users\fan_shuyuan\Desktop\科研ing\工艺流程优化\HPBO-V5\extractive_distillation\extractive_distillation_BOBYQA.bkp
{'target': None, 'params': None, 'constraint': None}
c:\Users\fan_shuyuan\Desktop\科研ing\工艺流程优化\HPBO-V5\extractive_distillation\extractive_distillation_BOBYQA.bkp
{'target': None, 'params': None, 'constraint': None}
c:\Users\fan_shuyuan\Desktop

In [ ]:
subprocess.run('taskkill /F /IM AspenPlus.exe')

CompletedProcess(args='taskkill /F /IM AspenPlus.exe', returncode=0)